In [54]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import adjusted_rand_score, silhouette_score
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.stem.wordnet import WordNetLemmatizer
#nltk.download('wordnet')
from nltk.stem import PorterStemmer
stop_words = set(stopwords.words('english'))

In [55]:
kleister = pd.read_csv('/home/becode/AI/Data/Faktion/kleister-charity/test-A/in.tsv', sep='\t', names=['filename', 'keys', 'text_djvu', 'text_tesseract', 'text_textract', 'text_best'])
#print(kleister.head())
print(kleister.shape)
kleister = kleister.dropna()
print(kleister.shape)
kleister =kleister.drop(columns=['keys', 'text_djvu','text_textract','text_best'])
kleister['text_tesseract'] = kleister['text_tesseract'].astype(str)
kleister['text_tesseract'] = kleister['text_tesseract'].apply(lambda x: x.replace("\n",""))
kleister['text_tesseract'] = kleister['text_tesseract'].apply(lambda x: x.replace("\\n",""))

(609, 6)
(592, 6)


In [56]:
print(kleister['text_tesseract'][0])
print(f"\n{kleister['text_tesseract'][100]}")
print(f"\n{kleister['text_tesseract'][300]}")

0 r = - ’ AT \ PRAESTAT OPES SAPIENTIA HAMPTON SCHOOL(A Charitable Company Limited by Guarantee)REPORT AND FINANCIAL STATEMENTSFOR THE YEAR ENDED 31 AUGUST 2016Registered Company No: 06264434Registered Charity No: 1120005HAMPTON SCHOOLCONTENTS PageChairman’s Report 1Legal and administrative information 2-3Governors’ report 4-26Independent Auditor’s report 27Statement of financial activities for the year ended 31 August 2016 28Statement of financial activities for the year ended 31 August 2015 29Balance sheet 30Cashflow statement 31Notes to the financial statements 32-48HAMPTON SCHOOL - CHAIRMAN'S REPORT - FOR THE YEAR ENDED 31 AUGUST 2016I'm delighted to say that it has been another very successful year for our Trust and its Schools. This success was verypublicly recognised by the team from the Independent Schools Inspectorate (ISi) which inspected the Schools in March2016. Its Report highlighted the very many excellent areas of the Schools’ activities and particularly concluded forHam

In [57]:
# Tokenize sentence and word
def tokenize_lem(text):
    lem = WordNetLemmatizer()
    stem = PorterStemmer()
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.match('[a-zA-Z]', token) and not token in stop_words and len(token)>3: # changed from if re.search('[a-zA-Z]', token)
            #filtered_tokens.append(token)
            lemmatized_word = lem.lemmatize(token)#, 'v')
            #stemmed = stem.stem(token)
            filtered_tokens.append(lemmatized_word)
            #filtered_tokens.append(stemmed)
    return filtered_tokens

In [58]:
kleister['text_tesseract'] = kleister['text_tesseract'].apply(tokenize_lem)

In [59]:
print(kleister['text_tesseract'][0])
print(f"\n{kleister['text_tesseract'][100]}")
print(f"\n{kleister['text_tesseract'][300]}")

['praestat', 'opes', 'sapientia', 'hampton', 'school', 'charitable', 'company', 'limited', 'guarantee', 'report', 'financial', 'statementsfor', 'year', 'ended', 'august', 'company', 'charity', 'schoolcontents', 'pagechairman', 'report', 'administrative', 'information', 'report', 'auditor', 'report', 'financial', 'activity', 'year', 'ended', 'august', 'financial', 'activity', 'year', 'ended', 'august', 'sheet', 'statement', 'financial', 'statement', 'school', 'chairman', 'report', 'year', 'ended', 'august', 'delighted', 'another', 'successful', 'year', 'trust', 'school', 'success', 'verypublicly', 'recognised', 'team', 'independent', 'school', 'inspectorate', 'inspected', 'school', 'march2016', 'report', 'highlighted', 'many', 'excellent', 'area', 'school', 'activity', 'particularly', 'concluded', 'forhampton', 'school', 'quality', 'pupil', 'achievement', 'academic', 'co-curricular', 'area', 'exceptional', 'wenton', 'declare', 'quality', 'leadership', 'management', 'including', 'link', 

In [60]:
print(len(kleister['text_tesseract'][0]))
print(len(kleister['text_tesseract'][100]))
print(len(kleister['text_tesseract'][300]))

8145
1012
499


In [61]:
kleister['length'] = kleister['text_tesseract'].apply(lambda x : len(x))

In [62]:
print(kleister['length'].sort_values(ascending=False))

47     55096
429    37538
375    35383
56     21197
21     13807
       ...  
235      267
576      262
317      252
260      118
390       91
Name: length, Length: 592, dtype: int64


In [63]:
print(kleister['length'].unique())

[ 8145  8673   830   768  3292  1485   806  2250  5783  2648  2397  1327
   782  3394   783   719  7158  5205  3329  3280 13807  2478   732   923
  1309  1396  4545  4391  4126  4266  1183  2956  3427  2244  2548  1732
   745  8267   983  1094   518  2331   536  6050 55096   972  6850  8100
  2772  4991  5193  1776  1803 21197  4525   636  1174  3212  6666   765
  6347  1019  4434   862  1925  3193   837  1807   723  1371  5480  2630
  2825   953   701  2418   935   438  8034  1083  2104   801   963  1464
  5312  2456   733   602  4360  2391   279  4040  2046   776  2453  2610
  1012  3373   541  6605  4370  1362  4393  2107  1645  5129  1069  8720
  1384  1210  1757   874   707  1709  6746  1256  1585  6292  2644  1435
  6762  4368  1088  4071  4741  1908  4549   492  2642   358  1255  1235
  2048   533  4652  5001  4851  3447  1067  1428  6370  5684  9170  1604
  1385  3358  1086   544  3727  4304  2342  7497   498  1285  8768  2416
  1223  1836  1063  4451  1244  1175  2584   836  3

In [64]:
print(kleister['text_tesseract'].apply(lambda x : set(x)))

0      {cricketer, relation, isbeing, fullyfunded.-, ...
1      {strengthening, task, signedhurstpierpoint, re...
2      {valuedetails, relation, operated, print, addr...
3      {setout, relate, call, next, address, trustees...
4      {faithfulfinancial, isbeing, book, ofbishop, p...
                             ...                        
604    {setout, relation, prepared, treasurer, recomm...
605    {medium, strengthening, relation, providefeedb...
606    {setout, strengthening, relation, positiveenvi...
607    {wale, employed, prepared, detection, book, be...
608    {medium, setout, task, relation, isbeing, spel...
Name: text_tesseract, Length: 592, dtype: object


In [65]:
print(kleister['text_tesseract'].apply(lambda x : len(set(x))))

0      2995
1      2877
2       472
3       452
4      1699
       ... 
604     438
605    2160
606    3064
607     603
608    3506
Name: text_tesseract, Length: 592, dtype: int64


In [66]:
print(kleister['text_tesseract'].apply(lambda x : len(set(x))).max())

13896
